# Домашнее задание к лекции "Базовые понятия статистики"

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

In [359]:
import pandas as pd

### Задание 1. Загрузка данных

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names), загрузить его и оставить 8 столбцов для дальнейшего изучения: `surgery?`, `Age`, `rectal temperature`, `pulse`, `respiratory rate`, `temperature of extremities`, `pain`, `outcome`. 

In [360]:
na_values = ['?']
horse = pd.read_csv(r'horse_data.csv', header=None, na_values=na_values)

# surgery? - хирург.операция? - имеет один пропуск, зависит от 12
horse[0].fillna(horse.groupby([12])[0].transform('median'), inplace=True)

In [361]:
horse_v2 = horse.copy(deep=True) # делаем клон
horse_v2.drop([2,7,8,9,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27], axis=1, inplace=True) # удаляем лишнии столбцы
horse_v2.rename(columns={0: 'surgery?', 1: 'Age', 3: 'rectal_temperature', 4: 'pulse', 
                         5: 'respiratory_rate', 6: 'temperature_of_extremities', 10: 'pain', 
                         22: 'outcome'}, inplace=True)

### Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [362]:
horse_v2.describe()

,surgery?,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,300.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.400000,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490716,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


In [ ]:
# surgery? - выбросов нет
# Age - есть выбросы
# rectal temperature - выбросов нет
# pulse - есть выбросы
# respiratory rate - есть выбросы
# temperature of extremities - выбросов нет
# pain - выбросов нет
# outcome - есть выбросы

### Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [363]:
horse_v2.isnull().sum()

surgery?                       0
Age                            0
rectal_temperature            60
pulse                         24
respiratory_rate              58
temperature_of_extremities    56
pain                          55
outcome                        1
dtype: int64

In [ ]:
# surgery? - был в начале один пропуск, устранил
# Age - пропусков нет
# rectal temperature - 60 пропусков
# pulse - 24 пропуска
# respiratory rate - 58 пропусков
# temperature of extremities - 56 пропусков
# pain - 55 пропусков
# outcome - 1 пропуск

In [365]:
# temperature of extremities - температура конечностей, 
# горячие конечности должны коррелировать с повышенной ректальной температурой - rectal_temperature.

horse_v2['rectal_temperature'].fillna(horse_v2.query('temperature_of_extremities == 3').groupby(['pulse'])['rectal_temperature'].transform('median'), inplace=True)
horse_v2['rectal_temperature'].fillna(horse_v2.query('temperature_of_extremities == 3').groupby(['temperature_of_extremities'])['rectal_temperature'].transform('median'), inplace=True)

# больше ни как не определить rectal_temperature, поэтому удаляем rectal_temperature с пропусками
horse_v2.dropna(subset=['rectal_temperature'], inplace=True)

# пульс имеет какую то зависимость, но это глубоко предметно, поэтому пропуски уадляем
horse_v2.dropna(subset=['pulse'], inplace=True)

# предполагаю, что исход зависит от 'temperature_of_extremities', 'pulse', 'surgery?'

horse_v2['outcome'].fillna(horse_v2.groupby(['temperature_of_extremities', 'pulse', 'surgery?'])['outcome'].transform('median'), inplace=True)

horse_v2.isnull().sum()

surgery?                       0
Age                            0
rectal_temperature             0
pulse                          0
respiratory_rate              36
temperature_of_extremities    42
pain                          43
outcome                        0
dtype: int64

In [366]:
# respiratory_rate - частота дыхания будем определять на основание 'pain', 'rectal_temperature', 'pulse' через медиану

horse_v2['respiratory_rate'].fillna(horse_v2.groupby(['Age', 'pain', 'rectal_temperature', 'pulse'])['respiratory_rate'].transform('median'), inplace=True)
horse_v2['respiratory_rate'].fillna(horse_v2.groupby(['Age', 'pain', 'rectal_temperature'])['respiratory_rate'].transform('median'), inplace=True)
horse_v2['respiratory_rate'].fillna(horse_v2.groupby(['Age', 'pain','pulse'])['respiratory_rate'].transform('median'), inplace=True)
horse_v2['respiratory_rate'].fillna(horse_v2.groupby(['Age', 'rectal_temperature', 'pulse'])['respiratory_rate'].transform('median'), inplace=True)
horse_v2['respiratory_rate'].fillna(horse_v2.groupby(['Age', 'rectal_temperature'])['respiratory_rate'].transform('median'), inplace=True)
horse_v2['respiratory_rate'].fillna(horse_v2.groupby(['Age', 'pulse'])['respiratory_rate'].transform('median'), inplace=True)

horse_v2.isnull().sum()

surgery?                       0
Age                            0
rectal_temperature             0
pulse                          0
respiratory_rate               0
temperature_of_extremities    42
pain                          43
outcome                        0
dtype: int64

In [367]:
# горячие конечности должны коррелировать с повышенной ректальной температурой - rectal_temperature.

horse_v2['temperature_of_extremities'].fillna(horse_v2.groupby(['rectal_temperature'])['temperature_of_extremities'].transform('median').round(), inplace=True)
horse_v2['temperature_of_extremities'].fillna(horse_v2['temperature_of_extremities'].median(), inplace=True)

horse_v2.isnull().sum()

surgery?                       0
Age                            0
rectal_temperature             0
pulse                          0
respiratory_rate               0
temperature_of_extremities     0
pain                          43
outcome                        0
dtype: int64

In [368]:
# боль определить на основании моих знаний не возможно. Будем предполагать

horse_v2['pain'].fillna(horse_v2.groupby(['rectal_temperature','pulse','respiratory_rate','outcome'])['pain'].transform('median').round(), inplace=True)
horse_v2['pain'].fillna(horse_v2.groupby(['rectal_temperature','pulse','respiratory_rate'])['pain'].transform('median').round(), inplace=True)
horse_v2['pain'].fillna(horse_v2.groupby(['rectal_temperature','respiratory_rate'])['pain'].transform('median').round(), inplace=True)
horse_v2['pain'].fillna(horse_v2.groupby(['respiratory_rate','outcome'])['pain'].transform('median').round(), inplace=True)
horse_v2['pain'].fillna(1, inplace=True)

horse_v2.isnull().sum()

surgery?                      0
Age                           0
rectal_temperature            0
pulse                         0
respiratory_rate              0
temperature_of_extremities    0
pain                          0
outcome                       0
dtype: int64